# Frame Extraction: Abilities, Kill Feed, Map

#### Goal: Extracting Regions of Interest (ROI) from all video feeds in order to total 20,000 individual frames for each section of the screen

Uses opencv-python to take in local video files, and then takes the config information to create different frames of important aspects of the screen

In [1]:
import os
import cv2
import json
from tqdm import tqdm

In [3]:
f = open('../../config/segmentation-information.json')
segmentation_section_data = json.load(f)
f.close()

In [5]:
## CREATING FRAME DATA
video_path = '../../../valo-gameplay-1080-sage.mp4'

cap = cv2.VideoCapture(video_path)
total_frame_count = cap.get(cv2.CAP_PROP_FRAME_COUNT)

# Create target count so that we create 20,000 individual frames for each dataset
target_frame_count = 20000
frame_count = 0
file_count = 1
frame_interval = max(1, total_frame_count / target_frame_count)

for _ in tqdm(range(int(total_frame_count))):
    ret, frame = cap.read()
    if not ret:
        break
    
    if frame_count % 1 < frame_interval:
        for _,segment in segmentation_section_data.items():
            # Collect ROI screen percentage data from config file
            height, width, _ = frame.shape
            x1_percentage, x2_percentage = segment["width_percentages"]
            y1_percentage, y2_percentage = segment["height_percentages"]
            
            # Create output folder if doesn't exist in data and create final file destination
            output_folder = segment["output_folder"]
            if not os.path.exists("../data/" + output_folder):
                os.makedirs("../data/" + output_folder)
            frame_filename = f"../data/{output_folder}frame_{int(file_count):04d}.jpg"
            
            # Translate screen percentages to actual pixel data to create ROI
            x1 = int(width * x1_percentage / 100)
            x2 = int(width * x2_percentage / 100)
            y1 = int(height * y1_percentage / 100)
            y2 = int(height * y2_percentage / 100)
            
            # Create ROI and then write to file
            ROI = frame[y1:y2,x1:x2,:]
            cv2.imwrite(frame_filename, ROI)
        
    frame_count += frame_interval
    file_count += 1

cap.release()
cv2.destroyAllWindows()

100%|█████████████████████████████████████| 37636/37636 [07:09<00:00, 87.66it/s]
